In [1]:
import joblib
import pandas as pd
import numpy as np


In [2]:

model = joblib.load(r'D:\xgboost_model.joblib')
models = joblib.load(r'D:\interval_models.joblib')
scaler = joblib.load( r'D:\model_scaler.joblib')
price_scaler = joblib.load( r'D:\price_model_scaler.joblib')
X_test = joblib.load(r'D:\X_test.joblib')

In [3]:
print(model)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.6, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.3, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, objective='reg:tweedie', ...)


In [4]:
data = pd.read_excel(r'C:\Users\Yuva S Korlaband\Downloads\VCM ML Model\VCM ML Model\VCM_Price_Inputs.xlsx')
data['Vintage Year'] = pd.to_datetime(data['Vintage Year'], errors='coerce')
data['Delivery_Year'] = pd.to_datetime(data['Delivery_Year'], errors='coerce')
data['Date_of_Trade'] = pd.to_datetime(data['Date_of_Trade'], errors='coerce')
data['Vintage Year'] = data['Vintage Year'].dt.year
data['Delivery_Year'] = data['Delivery_Year'].dt.year
data['Date_of_Trade'] = (data['Date_of_Trade'] - pd.to_datetime('2005-01-01')).dt.days

numeric_columns2 = ['Transaction_Volume', 'Date_of_Trade', 'Delivery_Year', 'Vintage Year']

for index, row in data.iterrows():
    row_subset = row[:8]
    row_dict = row_subset.to_dict()

    for col in ['Protocol_Type', 'Verification_Standard', 'Region', 'Market Segment']:
        new_col_name = f"{col}_{row_dict[col]}"
        row_dict[new_col_name] = 1

    input_data = pd.DataFrame([row_dict])

    input_df = pd.DataFrame(input_data)

    # Assuming you have a 'scaler' object for scaling
    input_df[numeric_columns2] = scaler.transform(input_df[numeric_columns2])

    missing_columns = set(X_test.columns) - set(input_df.columns)

    for col in missing_columns:
        input_df[col] = 0

    X_test_columns_order = X_test.columns.tolist()
    input_df = input_df[X_test_columns_order]

    # Predict using the main model
    price = model.predict(input_df)
    price_scale_reshaped = price.reshape(1, -1)
    predicted_prices = price_scaler.inverse_transform(price_scale_reshaped)
    data.at[index, 'Price_of_Trade'] = predicted_prices[0][0]

    # Predict using the list of 100 models for interval
    predicted_prices = []

    for i, model_interval in enumerate(models):
        predicted_price = model_interval.predict(input_df)
        predicted_prices.append(predicted_price)

    predicted_prices = np.array(predicted_prices)
    inverse_scaled_prices = price_scaler.inverse_transform(predicted_prices)

    percentile_5 = np.percentile(inverse_scaled_prices, 5)
    percentile_95 = np.percentile(inverse_scaled_prices, 95)

    min_price = np.min(inverse_scaled_prices)
    max_price = np.max(inverse_scaled_prices)

    data.at[index, 'lower_interval'] = min_price
    data.at[index, 'upper_interval'] = max_price

data['Date_of_Trade'] = pd.to_timedelta(data['Date_of_Trade'], unit='D') + pd.Timestamp('2005-01-01')
data['Date_of_Trade'] = data['Date_of_Trade'].dt.strftime('%d-%m-%Y')
vcm_json_output = data.to_json(orient='records')

In [5]:
print(vcm_json_output)

[{"Date_of_Trade":"27-10-2023","Transaction_Volume":50000,"Delivery_Year":2023,"Protocol_Type":"REDD+","Verification_Standard":"VCS","Region":"South America","Vintage Year":2018,"Market Segment":"Retail","Price_of_Trade":7.6119394302,"lower_interval":5.2806181908,"upper_interval":11.64265728},{"Date_of_Trade":"27-10-2022","Transaction_Volume":50000,"Delivery_Year":2022,"Protocol_Type":"REDD+","Verification_Standard":"VCS","Region":"South America","Vintage Year":2017,"Market Segment":"Retail","Price_of_Trade":11.8684959412,"lower_interval":9.5319824219,"upper_interval":14.0277175903},{"Date_of_Trade":"27-10-2023","Transaction_Volume":500000,"Delivery_Year":2023,"Protocol_Type":"REDD+","Verification_Standard":"VCS","Region":"South America","Vintage Year":2018,"Market Segment":"Retail","Price_of_Trade":7.179731369,"lower_interval":4.6850543022,"upper_interval":11.3432655334},{"Date_of_Trade":"27-10-2023","Transaction_Volume":50000,"Delivery_Year":2023,"Protocol_Type":"REDD+","Verification

In [12]:
def vcm_model(df):

    
    model = joblib.load(r'D:\xgboost_model.joblib')
    models = joblib.load(r'D:\interval_models.joblib')
    scaler = joblib.load( r'D:\model_scaler.joblib')
    price_scaler = joblib.load( r'D:\price_model_scaler.joblib')
    X_test = joblib.load(r'D:\X_test.joblib')
    
    
    data = df.drop(columns=['ID'])
    data['Vintage Year'] = pd.to_datetime(data['Vintage Year'], errors='coerce')
    data['Delivery_Year'] = pd.to_datetime(data['Delivery_Year'], errors='coerce')
    data['Date_of_Trade'] = pd.to_datetime(data['Date_of_Trade'], errors='coerce')
    data['Vintage Year'] = data['Vintage Year'].dt.year
    data['Delivery_Year'] = data['Delivery_Year'].dt.year
    data['Date_of_Trade'] = (data['Date_of_Trade'] - pd.to_datetime('2005-01-01')).dt.days

    numeric_columns2 = ['Transaction_Volume', 'Date_of_Trade', 'Delivery_Year', 'Vintage Year']

    for index, row in data.iterrows():
        row_subset = row[:8]
        row_dict = row_subset.to_dict()

        for col in ['Protocol_Type', 'Verification_Standard', 'Region', 'Market Segment']:
            new_col_name = f"{col}_{row_dict[col]}"
            row_dict[new_col_name] = 1

        input_data = pd.DataFrame([row_dict])

        input_df = pd.DataFrame(input_data)

        # Assuming you have a 'scaler' object for scaling
        input_df[numeric_columns2] = scaler.transform(input_df[numeric_columns2])

        missing_columns = set(X_test.columns) - set(input_df.columns)

        for col in missing_columns:
            input_df[col] = 0

        X_test_columns_order = X_test.columns.tolist()
        input_df = input_df[X_test_columns_order]

        # Predict using the main model
        price = model.predict(input_df)
        price_scale_reshaped = price.reshape(1, -1)
        predicted_prices = price_scaler.inverse_transform(price_scale_reshaped)
        data.at[index, 'Price_of_Trade'] = predicted_prices[0][0]

        # Predict using the list of 100 models for interval
        predicted_prices = []

        for i, model_interval in enumerate(models):
            predicted_price = model_interval.predict(input_df)
            predicted_prices.append(predicted_price)

        predicted_prices = np.array(predicted_prices)
        inverse_scaled_prices = price_scaler.inverse_transform(predicted_prices)

        percentile_5 = np.percentile(inverse_scaled_prices, 5)
        percentile_95 = np.percentile(inverse_scaled_prices, 95)

        min_price = np.min(inverse_scaled_prices)
        max_price = np.max(inverse_scaled_prices)

        data.at[index, 'lower_interval'] = percentile_5
        data.at[index, 'upper_interval'] = percentile_95

    data['Date_of_Trade'] = pd.to_timedelta(data['Date_of_Trade'], unit='D') + pd.Timestamp('2005-01-01')
    data['Date_of_Trade'] = data['Date_of_Trade'].dt.strftime('%d-%m-%Y')
    return data

In [30]:
df = pd.read_excel(r'C:\Users\Yuva S Korlaband\Downloads\VCM ML Model\VCM ML Model\VCM_Price_Inputs.xlsx')
data = [
    {
        "ID":1,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":2,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    }
]
#df = pd.DataFrame(data)
print(df.head())
result = vcm_model(df)

result["Delivery_Year"] = df["Delivery_Year"].astype(str)
result["Vintage Year"] = df["Vintage Year"].astype(str)
result.insert(0, "ID", df["ID"])
vcm_json_output = result.to_json(orient='records')

   ID Date_of_Trade  Transaction_Volume Delivery_Year Protocol_Type  \
0   1    2023-10-27               50000    2023-10-27         REDD+   
1   2    2022-10-27               50000    2022-10-27         REDD+   
2   3    2023-10-27              500000    2023-10-27         REDD+   
3   4    2023-10-27               50000    2023-10-27         REDD+   
4   5    2023-10-27               50000    2023-10-27         REDD+   

  Verification_Standard         Region Vintage Year Market Segment  \
0                   VCS  South America   2018-12-31         Retail   
1                   VCS  South America   2017-12-21         Retail   
2                   VCS  South America   2018-12-31         Retail   
3                   VCS  South America   2018-12-31      Wholesale   
4                   VCS         Africa   2018-12-31         Retail   

   Price_of_Trade  lower_interval  upper_interval  
0             NaN             NaN             NaN  
1             NaN             NaN             Na

In [31]:
print(vcm_json_output)

[{"ID":1,"Date_of_Trade":"27-10-2023","Transaction_Volume":50000,"Delivery_Year":"2023-10-27","Protocol_Type":"REDD+","Verification_Standard":"VCS","Region":"South America","Vintage Year":"2018-12-31","Market Segment":"Retail","Price_of_Trade":7.6119394302,"lower_interval":6.2259691238,"upper_interval":10.4930246353},{"ID":2,"Date_of_Trade":"27-10-2022","Transaction_Volume":50000,"Delivery_Year":"2022-10-27","Protocol_Type":"REDD+","Verification_Standard":"VCS","Region":"South America","Vintage Year":"2017-12-21","Market Segment":"Retail","Price_of_Trade":11.8684959412,"lower_interval":10.5266553879,"upper_interval":13.2524194717},{"ID":3,"Date_of_Trade":"27-10-2023","Transaction_Volume":500000,"Delivery_Year":"2023-10-27","Protocol_Type":"REDD+","Verification_Standard":"VCS","Region":"South America","Vintage Year":"2018-12-31","Market Segment":"Retail","Price_of_Trade":7.179731369,"lower_interval":5.9090361834,"upper_interval":9.7852848053},{"ID":4,"Date_of_Trade":"27-10-2023","Transa

# request test


In [1]:
import requests
import json
import time


In [2]:
start_time = time.time()
url = "https://importbrokersheet2.azurewebsites.net/api/v3/httpstrigger?token=$2b$12$tmKI50PxbL8cK07G/9hyF.XdUdiwTRXgXb8bQ/LXksMiAXldqV4zK"

data = [
    {
        "ID":14,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":2123,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":2672,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":2455,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":271,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":266,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":254,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":237,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":17,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":21265,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":267234,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":24525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":27643,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2625,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":236523,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    
    {
        "ID":14,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":2123,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":2672,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":2455,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":271,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":266,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":254,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":237,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":17,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":21265,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":267234,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":24525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":27643,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2625,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":236523,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":14,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":2123,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":2672,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":2455,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":271,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":266,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":254,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":237,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":17,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":21265,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":267234,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":24525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":27643,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2625,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":236523,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":14,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":2123,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":2672,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":2455,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":271,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":266,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":254,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":237,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":17,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":21265,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":267234,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":24525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":27643,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2625,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":236523,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":14,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":2123,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":2672,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":2455,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":271,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":266,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":254,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":237,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":17,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":21265,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":267234,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":24525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":27643,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2625,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":236523,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":14,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":2123,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":2672,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":2455,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":271,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":266,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":254,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":237,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":17,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":21265,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":267234,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":24525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":27643,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2625,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":236523,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":14,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":2123,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":2672,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":2455,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":271,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":266,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":254,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":237,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":17,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":21265,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":267234,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":24525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":27643,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2625,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":236523,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },{
        "ID":14,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":2123,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":2672,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":2455,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":271,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":266,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":254,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":237,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":17,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":21265,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":267234,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":24525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":27643,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2625,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":236523,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":14,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":2123,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":2672,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":2455,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":271,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":266,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":254,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":237,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":17,
        "Date_of_Trade": "2023-11-30",
        "Transaction_Volume": 100000,
        "Delivery_Year": "27-10-2023",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "27-10-2023",
        "Market Segment": "Wholesale"
    },
    {
        "ID":21265,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2018",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2018",
        "Market Segment": "Retail"
    },
    {
        "ID":267234,
        "Date_of_Trade": "2022-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2017",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2008",
        "Market Segment": "Retail"
    },
    {
        "ID":24525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2016",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":27643,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2014",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2625,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2015",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":2525,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2013",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    },
    {
        "ID":236523,
        "Date_of_Trade": "2023-12-01",
        "Transaction_Volume": 150000,
        "Delivery_Year": "31-12-2012",
        "Protocol_Type": "Renewable Energy",
        "Verification_Standard": "VCS",
        "Region": "Asia",
        "Vintage Year": "31-12-2017",
        "Market Segment": "Retail"
    }
]

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, data=json.dumps(data), headers=headers)

print(response.status_code)
print(response.text)

end_time = time.time() 
duration = end_time - start_time 

print(f"total Time: {duration} seconds")

200
[{"ID":14,"Date_of_Trade":"30-11-2023","Transaction_Volume":100000,"Delivery_Year":"27-10-2023","Protocol_Type":"Renewable Energy","Verification_Standard":"VCS","Region":"Asia","Vintage Year":"27-10-2023","Market Segment":"Wholesale","Price_of_Trade":2.9826376438,"lower_interval":2.1161759734,"upper_interval":3.7694423079},{"ID":2123,"Date_of_Trade":"01-12-2023","Transaction_Volume":150000,"Delivery_Year":"31-12-2018","Protocol_Type":"Renewable Energy","Verification_Standard":"VCS","Region":"Asia","Vintage Year":"31-12-2018","Market Segment":"Retail","Price_of_Trade":4.51917696,"lower_interval":3.2332323432,"upper_interval":5.7040956497},{"ID":2672,"Date_of_Trade":"01-12-2022","Transaction_Volume":150000,"Delivery_Year":"31-12-2017","Protocol_Type":"Renewable Energy","Verification_Standard":"VCS","Region":"Asia","Vintage Year":"31-12-2008","Market Segment":"Retail","Price_of_Trade":3.790127039,"lower_interval":3.2158473134,"upper_interval":4.7211117506},{"ID":2455,"Date_of_Trade":"